# MCP + LangGraph 핸즈온 튜토리얼

- 작성자: [테디노트](https://youtube.com/c/teddynote)
- 강의: [패스트캠퍼스 RAG 비법노트](https://fastcampus.co.kr/data_online_teddy)

**참고자료**
- https://modelcontextprotocol.io/introduction
- https://github.com/langchain-ai/langchain-mcp-adapters

## 환경설정

아래 설치 방법을 참고하여 `uv` 를 설치합니다.

**uv 설치 방법**

```bash
# macOS/Linux
curl -LsSf https://astral.sh/uv/install.sh | sh

# Windows (PowerShell)
irm https://astral.sh/uv/install.ps1 | iex
```

**의존성 설치**

```bash
uv pip install -r requirements.txt
```

환경변수를 가져옵니다.

In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv(override=True)
logging.langsmith("LangGraph MCP Adapters HandsOn")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangGraph MCP Adapters HandsOn


## MultiServerMCPClient

사전에 `mcp_server_remote.py` 를 실행해둡니다. 터미널을 열고 가상환경이 활성화 되어 있는 상태에서 서버를 실행해 주세요.

> 명령어
```bash
source .venv/bin/activate
python mcp_server_remote.py
```

`async with` 로 일시적인 Session 연결을 생성 후 해제

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_teddynote.messages import ainvoke_graph, astream_graph
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

async with MultiServerMCPClient(
    {
        "weather": {
            # 서버의 포트와 일치해야 합니다.(8005번 포트)
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
) as client:
    print(client.get_tools())
    agent = create_react_agent(model, client.get_tools())
    answer = await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x11c5b39c0>)]

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
서울의 날씨 정보를 확인해 드리겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in 서울
🔄 Node: agent 🔄
- - - - - - - - - - - - - -

다음의 경우에는 session 이 닫혔기 때문에 도구에 접근할 수 없는 것을 확인할 수 있습니다.

In [3]:
await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
서울의 날씨 정보를 확인해 드리겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Error: ClosedResourceError()
 Please fix your mistakes.
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
죄송합니다. 기술적인 문제가 발생했습니다. 다시 서울의 날씨 정보를 확인해 보겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Error: ClosedResourceError()
 Please fix your mistakes.
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
죄송합니다만, 현재 날씨 정보 서비스에 접속하는 데 문제가 있는 것 같습니다. 시스템 오류로 인해 서울의 날씨 정보를 제공해 드릴 수 없습니다. 나중에 다시 시도해 주시거나, 다른 날씨 서비스를 이용해 보시기 바랍니다.

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-0c876e8f-2cbf-4a5e-b81b-731cf8995079', usage_metadata={'input_tokens': 0, 'output_tokens': 132, 'total_tokens': 132, 'input_token_details': {}}),
 'metadata': {'langgraph_step': 5,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:7d5441d0-5482-1867-4a0a-6b773e4716f7',
  'checkpoint_ns': 'agent:7d5441d0-5482-1867-4a0a-6b773e4716f7',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

이제 그럼 Async Session 을 유지하며 도구에 접근하는 방식으로 변경해 보겠습니다.

In [4]:
# 1. 클라이언트 생성
client = MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
)


# 2. 명시적으로 연결 초기화 (이 부분이 필요함)
# 초기화
await client.__aenter__()

# 이제 도구가 로드됨
print(client.get_tools())  # 도구가 표시됨

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x11c5b3100>)]


langgraph 의 에이전트를 생성합니다.

In [5]:
# 에이전트 생성
agent = create_react_agent(model, client.get_tools())

그래프를 실행하여 결과를 확인합니다.

In [6]:
await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
서울의 날씨 정보를 확인해 드리겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in 서울
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
서울의 현재 날씨는 맑음입니다. "It's always Sunny in 서울"이라고 나와있네요. 즉, 서울은 현재 맑은 날씨를 보이고 있습니다.

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-042cc4b9-744d-41a9-961f-7e9b8eb164c6', usage_metadata={'input_tokens': 0, 'output_tokens': 74, 'total_tokens': 74, 'input_token_details': {}}),
 'metadata': {'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:d16fa451-709a-18fe-b578-1ea89d702204',
  'checkpoint_ns': 'agent:d16fa451-709a-18fe-b578-1ea89d702204',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

## Stdio 통신 방식

Stdio 통신 방식은 로컬 환경에서 사용하기 위해 사용합니다.

- 통신을 위해 표준 입력/출력 사용

참고: 아래의 python 경로는 수정하세요!

In [7]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_anthropic import ChatAnthropic

# Anthropic의 Claude 모델 초기화
model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

# StdIO 서버 파라미터 설정
# - command: Python 인터프리터 경로
# - args: 실행할 MCP 서버 스크립트
server_params = StdioServerParameters(
    command="./.venv/bin/python",
    args=["mcp_server_local.py"],
)

# StdIO 클라이언트를 사용하여 서버와 통신
async with stdio_client(server_params) as (read, write):
    # 클라이언트 세션 생성
    async with ClientSession(read, write) as session:
        # 연결 초기화
        await session.initialize()

        # MCP 도구 로드
        tools = await load_mcp_tools(session)
        print(tools)

        # 에이전트 생성
        agent = create_react_agent(model, tools)

        # 에이전트 응답 스트리밍
        await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x11e154220>)]

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
서울의 날씨 정보를 확인해 드리겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in 서울
🔄 Node: agent 🔄
- - - - - - - - - - - - - -

## RAG 를 구축한 MCP 서버 사용

- 파일: `mcp_server_rag.py`

사전에 langchain 으로 구축한 `mcp_server_rag.py` 파일을 사용합니다.

stdio 통신 방식으로 도구에 대한 정보를 가져옵니다. 여기서 도구는 `retriever` 도구를 가져오게 되며, 이 도구는 `mcp_server_rag.py` 에서 정의된 도구입니다. 이 파일은 사전에 서버에서 실행되지 **않아도** 됩니다.

In [8]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic
from langchain_teddynote.messages import astream_graph

# Anthropic의 Claude 모델 초기화
model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

# RAG 서버를 위한 StdIO 서버 파라미터 설정
server_params = StdioServerParameters(
    command="./.venv/bin/python",
    args=["./mcp_server_rag.py"],
)

# StdIO 클라이언트를 사용하여 RAG 서버와 통신
async with stdio_client(server_params) as (read, write):
    # 클라이언트 세션 생성
    async with ClientSession(read, write) as session:
        # 연결 초기화
        await session.initialize()

        # MCP 도구 로드 (여기서는 retriever 도구)
        tools = await load_mcp_tools(session)

        # 에이전트 생성 및 실행
        agent = create_react_agent(model, tools)

        # 에이전트 응답 스트리밍
        await astream_graph(
            agent, {"messages": "삼성전자가 개발한 생성형 AI의 이름을 검색해줘"}
        )


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
삼성전자가 개발한 생성형 AI의 이름을 검색해 드리겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이

## SSE 방식과 StdIO 방식 혼합 사용

- 파일: `mcp_server_rag.py` 는 StdIO 방식으로 통신
- `langchain-dev-docs` 는 SSE 방식으로 통신

SSE 방식과 StdIO 방식을 혼합하여 사용합니다.

In [9]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic

# Anthropic의 Claude 모델 초기화
model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

# 1. 다중 서버 MCP 클라이언트 생성
client = MultiServerMCPClient(
    {
        "document-retriever": {
            "command": "./.venv/bin/python",
            # mcp_server_rag.py 파일의 절대 경로로 업데이트해야 합니다
            "args": ["./mcp_server_rag.py"],
            # stdio 방식으로 통신 (표준 입출력 사용)
            "transport": "stdio",
        },
        "langchain-dev-docs": {
            # SSE 서버가 8765 포트에서 실행 중인지 확인하세요
            "url": "http://teddynote.io:8765/sse",
            # SSE(Server-Sent Events) 방식으로 통신
            "transport": "sse",
        },
    }
)


# 2. 비동기 컨텍스트 매니저를 통한 명시적 연결 초기화
await client.__aenter__()

Error in sse_reader: 


langgraph 의 `create_react_agent` 를 사용하여 에이전트를 생성합니다.

In [10]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

prompt = (
    "You are a smart agent. "
    "Use `retriever` tool to search on AI related documents and answer questions."
    "Use `langchain-dev-docs` tool to search on langchain / langgraph related documents and answer questions."
    "Answer in Korean."
)
agent = create_react_agent(
    model, client.get_tools(), prompt=prompt, checkpointer=MemorySaver()
)

구축해 놓은 `mcp_server_rag.py` 에서 정의한 `retriever` 도구를 사용하여 검색을 수행합니다.

In [11]:
config = RunnableConfig(recursion_limit=30, thread_id=1)
await astream_graph(
    agent,
    {
        "messages": "`retriever` 도구를 사용해서 삼성전자가 개발한 생성형 AI 이름을 검색해줘"
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
삼성전자가 개발한 생성형 AI 이름을 검색하기 위해 `retriever` 도구를 사용하겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-8676ef73-fc37-41ee-8cef-c8ae5177773c', usage_metadata={'input_tokens': 0, 'output_tokens': 340, 'total_tokens': 340, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:f962e1c5-2490-1395-093d-be9a181da22e',
  'checkpoint_ns': 'agent:f962e1c5-2490-1395-093d-be9a181da22e',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

이번에는 `langchain-dev-docs` 도구를 사용하여 검색을 수행합니다.

In [12]:
config = RunnableConfig(recursion_limit=30, thread_id=1)
await astream_graph(
    agent,
    {"messages": "langgraph-dev-docs 참고해서 self-rag 의 정의에 대해서 알려줘"},
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
LangGraph 개발 문서를 참고하여 Self-RAG의 정의에 대해 알아보겠습니다. 먼저 관련 문서를 검색해보겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
["Below is the list of references for the answer to the question.(Ordered by relevance descending):", "1. https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_adaptive_rag/", "2. https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_self_rag/", "3. https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_crag/", "4. https://langchain-ai.github.io/langgraph/tutorials/code_assistant/langgraph_code_assistant/"]
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Self-RAG에 대한 정의를 찾기 위해 두 번째 링크를 확인해보겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Self-RAG






[Skip to content](#self-rag)

**Join us at  [Interrupt: The Agent AI Conference by LangChain](https://interrupt.langchain.com/) on May 13 & 14 in San Francisco!**


{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-661748dd-399a-45bc-bd3f-ad79e848afa3', usage_metadata={'input_tokens': 0, 'output_tokens': 900, 'total_tokens': 900, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 12,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:d287bf8b-782b-939a-f554-acfe7936af14',
  'checkpoint_ns': 'agent:d287bf8b-782b-939a-f554-acfe7936af14',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

`MemorySaver` 를 사용하여 단기 기억을 유지합니다. 따라서, multi-turn 대화도 가능합니다.

In [13]:
await astream_graph(
    agent, {"messages": "이전의 내용을 bullet point 로 요약해줘"}, config=config
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
# Self-RAG 정의 요약

## 기본 개념
* Self-RAG는 Self-Reflective Retrieval Augmented Generation의 약자
* 검색 기반 생성(RAG) 전략에 자기 성찰 및 자기 평가 메커니즘을 통합한 방식
* 2023년 발표된 논문 "Self-RAG: Learning to Retrieve, Generate, and Critique through Self-Reflection"에서 제안됨

## 주요 특징
* 검색된 문서와 생성된 내용에 대한 자동 평가 시스템 포함
* 검색 여부, 문서 관련성, 답변 품질을 스스로 판단하는 의사결정 프로세스
* 환각(hallucination) 감지 및 방지 메커니즘 내장

## 평가 단계
* 검색 결정: 검색기를 사용할지 여부 결정 (예/아니오/계속)
* 문서 관련성 평가: 검색된 문서가 질문과 관련 있는지 판단 (관련 있음/없음)
* 환각 평가: 생성된 답변이 검색된 문서에 근거하는지 판단 (완전 지원/부분 지원/지원 없음)
* 유용성 평가: 생성된 답변이 질문에 유용한지 평가 (5점 척도)

## 작동 방식
* 질문 입력 → 문서 검색 → 문서 관련성 평가 → 답변 생성 → 답변 품질 평가
* 관련 문서가 없으면 질문 재구성(transform query) 수행
* 생성된 답변이 부적절하면 재생성 또는 질문 재구성 진행
* 자기 교정 루프를 통해 답변 품질 지속적 개선

## 장점
* 기존 RAG 시스템보다 더 정확하고 신뢰할 수 있는 답변 제공
* 검색된 정보의 관련성과 생성된 답변의 품질을 지속적으로 모니터링
* LangGraph를 사용하여 효과적으로 구현 가능

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-0d067c90-47f6-464f-8be7-7e76a8325691', usage_metadata={'input_tokens': 0, 'output_tokens': 678, 'total_tokens': 678, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 15,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:cdedffab-3e5e-c815-b714-d7ec5c9ad42f',
  'checkpoint_ns': 'agent:cdedffab-3e5e-c815-b714-d7ec5c9ad42f',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

## LangChain 에 통합된 도구 + MCP 도구

여기서는 LangChain 에 통합된 도구를 기존의 MCP 로만 이루어진 도구와 함께 사용이 가능한지 테스트 합니다.

In [14]:
from langchain_teddynote.tools.tavily import TavilySearch

# Tavily 검색 도구를 초기화 합니다. (news 타입, 최근 3일 내 뉴스)
tavily = TavilySearch(max_results=3, topic="news", days=3)

# 기존의 MCP 도구와 함께 사용합니다.
tools = client.get_tools() + [tavily]

langgraph 의 `create_react_agent` 를 사용하여 에이전트를 생성합니다.

In [15]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

prompt = "You are a smart agent with various tools. Answer questions in Korean."
agent = create_react_agent(model, tools, prompt=prompt, checkpointer=MemorySaver())

새롭게 추가한 `tavily` 도구를 사용하여 검색을 수행합니다.

In [16]:
await astream_graph(agent, {"messages": "오늘 뉴스 찾아줘"}, config=config)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
오늘 뉴스를 찾아드리겠습니다. 웹 검색을 통해 최신 뉴스를 검색하겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
[{"url": "https://racingnews365.com/lawson-handed-second-arrogance-evaluation-after-red-bull-axe", "raw_content": "Published Time: 2025-03-29T12:00:00+01:00\nLawson handed second ‘arrogance’ evaluation after Red Bull axe | RacingNews365\nRN365 Podcast Red Bull emergency meeting as Ferrari review disqualifications\nLogin\nHome\nF1 news\nCalendar\nStandings\nHamilton\nFull menu\nHome\nF1 news\nF1 Calendar\nF1 Standings\nInterviews\nVideos\nPodcast\nFormula E\nClose More\nHome\nF1 news\nF1 calendar 2025\nF1 standings 2025\nF1 drivers\nF1 teams\nF1 circuits\nF1 results\nF1 Live\nF1 videos\nF1 podcast\nFormula E\nClose menu\nWelcome at RacingNews365\nBecome part of the largest racing community in the United Kingdom. Create your free account now!\nShare your thoughts and opinions about F1\nWin fantastic prizes\nGet ac

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-8ce6e776-e95b-49b0-820b-2ca726de9724', usage_metadata={'input_tokens': 0, 'output_tokens': 718, 'total_tokens': 718, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:5dec81d8-79fe-570e-6d49-4673b5b5bef1',
  'checkpoint_ns': 'agent:5dec81d8-79fe-570e-6d49-4673b5b5bef1',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

`retriever` 도구가 원활하게 작동하는 것을 확인할 수 있습니다.

In [17]:
await astream_graph(
    agent,
    {
        "messages": "`retriever` 도구를 사용해서 삼성전자가 개발한 생성형 AI 이름을 검색해줘"
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
삼성전자가 개발한 생성형 AI 이름을 검색해보겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-cf796851-d1b6-4046-a4e9-2bd8cc6ccd54', usage_metadata={'input_tokens': 0, 'output_tokens': 428, 'total_tokens': 428, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 8,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:83468697-2121-88e1-cd4f-55a28286718b',
  'checkpoint_ns': 'agent:83468697-2121-88e1-cd4f-55a28286718b',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

## Smithery 에서 제공하는 MCP 서버

- 링크: https://smithery.ai/

사용한 도구 목록은 아래와 같습니다.

- Sequential Thinking: https://smithery.ai/server/@smithery-ai/server-sequential-thinking
  - 구조화된 사고 프로세스를 통해 역동적이고 성찰적인 문제 해결을 위한 도구를 제공하는 MCP 서버
- Desktop Commander: https://smithery.ai/server/@wonderwhy-er/desktop-commander
  - 다양한 편집 기능으로 터미널 명령을 실행하고 파일을 관리하세요. 코딩, 셸 및 터미널, 작업 자동화

**참고**

- smithery 에서 제공하는 도구를 JSON 형식으로 가져올때, 아래의 예시처럼 `"transport": "stdio"` 로 꼭 설정해야 합니다.

In [18]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic

# LLM 모델 초기화
model = ChatAnthropic(model="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000)

# 1. 클라이언트 생성
client = MultiServerMCPClient(
    {
        "server-sequential-thinking": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@smithery-ai/server-sequential-thinking",
                "--key",
                "89a4780a-53b7-4b7b-92e9-a29815f2669b",
            ],
            "transport": "stdio",  # stdio 방식으로 통신을 추가합니다.
        },
        "desktop-commander": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@wonderwhy-er/desktop-commander",
                "--key",
                "89a4780a-53b7-4b7b-92e9-a29815f2669b",
            ],
            "transport": "stdio",  # stdio 방식으로 통신을 추가합니다.
        },
        "document-retriever": {
            "command": "./.venv/bin/python",
            # mcp_server_rag.py 파일의 절대 경로로 업데이트해야 합니다
            "args": ["./mcp_server_rag.py"],
            # stdio 방식으로 통신 (표준 입출력 사용)
            "transport": "stdio",
        },
    }
)


# 2. 명시적으로 연결 초기화
await client.__aenter__()

langgraph 의 `create_react_agent` 를 사용하여 에이전트를 생성합니다.

In [19]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig


agent = create_react_agent(model, client.get_tools(), checkpointer=MemorySaver())

`Desktop Commander` 도구를 사용하여 터미널 명령을 실행합니다.

In [20]:
await astream_graph(
    agent,
    {
        "messages": "현재 경로를 포함한 하위 폴더 구조를 tree 로 그려줘. 단, .venv 폴더는 제외하고 출력해줘."
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
현재 경로와 하위 폴더 구조를 tree 형태로 그려드리겠습니다. `.venv` 폴더는 제외하고 출력하겠습니다.

이를 위해 `execute_command` 함수를 사용하여 tree 명령어를 실행하겠습니다. `-I` 옵션을 사용하면 특정 패턴을 제외할 수 있습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Command started with PID 5105
Initial output:
.
|____.git
| |____hooks
| |____info
| |____logs
| | |____refs
| | | |____heads
| | | |____remotes
| | | | |____origin
| |____objects
| | |____03
| | |____18
| | |____2d
| | |____36
| | |____3c
| | |____42
| | |____48
| | |____4d
| | |____59
| | |____5d
| | |____6f
| | |____75
| | |____93
| | |____96
| | |____99
| | |____9f
| | |____a7
| | |____a9
| | |____aa
| | |____b5
| | |____c0
| | |____c1
| | |____c8
| | |____cb
| | |____e2
| | |____e3
| | |____e4
| | |____e7
| | |____ea
| | |____f0
| | |____f7
| | |____info
| | |____pack
| |____refs
| | |____heads
| | |____remotes
| | | |____origin
| | |____tags
|____assets
|____data

🔄 Node: agent 🔄
- - - - - - - - - - - 

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-7325ff0e-cf97-471f-96ef-0ab2fc0ba215', usage_metadata={'input_tokens': 0, 'output_tokens': 321, 'total_tokens': 321, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 9,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:94e82fea-342b-2f07-08b0-432160e4029c',
  'checkpoint_ns': 'agent:94e82fea-342b-2f07-08b0-432160e4029c',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

이번에는 `Sequential Thinking` 도구를 사용하여 비교적 복잡한 작업을 수행할 수 있는지 확인합니다.

In [21]:
await astream_graph(
    agent,
    {
        "messages": (
            "`retriever` 도구를 사용해서 삼성전자가 개발한 생성형 AI 관련 내용을 검색하고 "
            "`Sequential Thinking` 도구를 사용해서 보고서를 작성해줘."
        )
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
삼성전자가 개발한 생성형 AI 관련 내용을 검색하고 보고서를 작성해드리겠습니다. 먼저 `retrieve` 도구를 사용하여 관련 정보를 검색한 후, `sequentialthinking` 도구를 활용해 체계적인 보고서를 작성하겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약,

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-467bb435-0acf-42a3-8d47-601b8775393a', usage_metadata={'input_tokens': 0, 'output_tokens': 2247, 'total_tokens': 2247, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 32,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:8e9e85a9-cd93-4316-29f0-663a875bec41',
  'checkpoint_ns': 'agent:8e9e85a9-cd93-4316-29f0-663a875bec41',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}